In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [1]:
%cd yolov5

D:\final_project\yolov5


In [3]:
!pip install -r requirements.txt

In [4]:
!pip install pandas requests tqdm

In [2]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [3]:
# 경로 설정
BASE_DIR = r"D:\final_project\VOC_PCB"
XML_DIR = os.path.join(BASE_DIR, "Annotations")
LABEL_DIR = os.path.join(BASE_DIR, "labels")  # 새로 생성될 폴더
IMAGE_DIR = os.path.join(BASE_DIR, "JPEGImages")

# 결함 유형 정의 (인덱스 순서 중요)
classes = ["short", "missing_hole", "mouse_bite", "open_circuit", "spurious_copper", "spur"]

# labels 폴더가 없으면 생성
if not os.path.exists(LABEL_DIR):
    os.makedirs(LABEL_DIR)

def convert_coordinates(size, box):
    """VOC 좌표를 YOLO 정규화 좌표로 변환"""
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return (x * dw, y * dh, w * dw, h * dh)

# XML 파일 전체 변환 작업
print("Converting XML annotations to YOLO format...")
xml_files = [f for f in os.listdir(XML_DIR) if f.endswith(".xml")]

for xml_file in tqdm(xml_files):
    tree = ET.parse(os.path.join(XML_DIR, xml_file))
    root = tree.getroot()
    
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    file_id = os.path.splitext(xml_file)[0]
    with open(os.path.join(LABEL_DIR, f"{file_id}.txt"), "w") as out_file:
        for obj in root.iter("object"):
            cls_name = obj.find("name").text
            if cls_name not in classes:
                continue
            
            cls_id = classes.index(cls_name)
            xmlbox = obj.find("bndbox")
            b = (float(xmlbox.find("xmin").text), float(xmlbox.find("xmax").text),
                 float(xmlbox.find("ymin").text), float(xmlbox.find("ymax").text))
            
            bb = convert_coordinates((width, height), b)
            # <class_id> <x_center> <y_center> <width> <height>
            out_file.write(f"{cls_id} {' '.join([f'{a:.6f}' for a in bb])}\n")

print("Conversion complete.")

Converting XML annotations to YOLO format...


100%|██████████████████████████████████████████████████████████████████████████| 10668/10668 [00:05<00:00, 1838.68it/s]

Conversion complete.


In [4]:
import yaml

data_yaml = {
    'path': BASE_DIR,
    'train': 'train.txt',
    'val': 'val.txt',
    'test': 'test.txt',
    'nc': 6,
    'names': classes
}

with open(os.path.join(BASE_DIR, 'dataset.yaml'), 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"dataset.yaml created at {BASE_DIR}")

dataset.yaml created at D:\final_project\VOC_PCB


In [5]:
# 현재 경로가 yolov5 폴더인지 다시 한번 확인
import os
print(f"현재 작업 경로: {os.getcwd()}")


현재 작업 경로: D:\final_project\yolov5


In [6]:
import os

# 경로 설정 (사용자 환경에 맞춤)
BASE_DIR = r"D:\final_project\VOC_PCB"
IMAGE_DIR = os.path.join(BASE_DIR, "JPEGImages")

# 수정할 리스트 파일들
target_files = ['train.txt', 'val.txt', 'test.txt']

for target in target_files:
    file_path = os.path.join(BASE_DIR, target)
    
    if not os.path.exists(file_path):
        print(f"파일을 찾을 수 없습니다: {file_path}")
        continue
        
    # 기존 파일 읽기
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # 경로 업데이트: 파일명 -> D:\final_project\VOC_PCB\JPEGImages\파일명.jpg
    new_lines = []
    for line in lines:
        name = line.strip()
        if not name:
            continue
        # 확장자가 붙어있지 않다면 .jpg를 붙여 절대 경로 생성
        if not name.endswith('.jpg'):
            full_path = os.path.join(IMAGE_DIR, f"{name}.jpg")
        else:
            full_path = os.path.join(IMAGE_DIR, name)
        new_lines.append(full_path + '\n')
    
    # 수정된 내용으로 덮어쓰기
    with open(file_path, 'w') as f:
        f.writelines(new_lines)

print("모든 경로가 절대 경로로 업데이트되었습니다.")

모든 경로가 절대 경로로 업데이트되었습니다.


In [8]:
import os
import glob

# 1. 경로 설정
BASE_DIR = r"D:\final_project\VOC_PCB"
# 폴더 이름을 'images'로 변경한 후의 경로
IMAGE_DIR = os.path.join(BASE_DIR, "images")

# 2. 기존 잘못된 캐시 파일(.cache) 삭제 (매우 중요)
cache_files = glob.glob(os.path.join(BASE_DIR, "*.cache"))
for cf in cache_files:
    try:
        os.remove(cf)
        print(f"삭제 완료: {cf}")
    except:
        pass

# 3. train.txt, val.txt, test.txt 내 경로를 새 images 폴더로 업데이트
target_files = ['train.txt', 'val.txt', 'test.txt']

for target in target_files:
    file_path = os.path.join(BASE_DIR, target)
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            lines = f.readlines()
        
        new_lines = []
        for line in lines:
            # 파일명만 추출해서 D:\final_project\VOC_PCB\images\파일명.jpg 로 재구성
            fname = os.path.basename(line.strip())
            if not fname.lower().endswith('.jpg'):
                fname += '.jpg'
            new_lines.append(os.path.join(IMAGE_DIR, fname) + '\n')
            
        with open(file_path, 'w') as f:
            f.writelines(new_lines)

print("데이터셋 경로 업데이트 및 캐시 삭제가 완료되었습니다.")

삭제 완료: D:\final_project\VOC_PCB\train.cache
데이터셋 경로 업데이트 및 캐시 삭제가 완료되었습니다.
